We need to get from mapped bam file to single fast5s:

Input is:
* Genome sequence Pgt21-0
* Fastq files to map with minimap. These were combined with "find * -type f -name '*.gz'| xargs -I X cat X >> 20210406.PgtGerminatedSpores.fastq.gz"
* Folder with multifast5 of fastqs mapped to the reference. These were combined with "find * -type f -name "*.fast5" | xargs -I X mv X $dir/." on the command line

Output:
* folder with single fast5s that are mapped

What we need to do:
* map all reads
* get ID of all mapped fastqs
* extract them in batches with the fast5_subset

In [1]:
import os

### INPUTS

### *****Define directories before running on different sampes*****

In [19]:
###define input directories here 
RefFn = '../../../resources/genomic_resources/chr_A_B_unassigned.fasta'
FastqFn = '../../../analysis/germinatedSpores/20210406.PgtGerminatedSpores.fastq.gz'
BamFn = '../../../analysis/germinatedSpores/20210406.Pgt210.PgtGerminatedSpores.sorted.bam'
InFast5Dir = '/media/ssd-01/ben/projects/Pgt210Methylation/rawData/germinatedSpores/fast5/combined'
OutDir = '../../../analysis/germinatedSpores/subsetFast5'
OutDirGuppy = '../../../analysis/germinatedSpores/subsetFast5/basecalled'

### *****Define directories before running on different sampes*****

In [20]:
RefFn = os.path.abspath(RefFn)
FastqFn = os.path.abspath(FastqFn)
BamFn = os.path.abspath(BamFn)
InFast5Dir = os.path.abspath(InFast5Dir)
OutDir = os.path.abspath(OutDir)
OutDirGuppy = os.path.abspath(OutDirGuppy)

In [4]:
threads = 8

In [7]:
##map all the reads with Minimap
!minimap2 -2 --sam-hit-only --secondary=no -t {threads} -ax map-ont {RefFn} {FastqFn} | samtools sort -@ {12-threads} -o {BamFn} -O bam -
!samtools index {BamFn}

[M::mm_idx_gen::2.808*1.49] collected minimizers
[M::mm_idx_gen::3.195*2.27] sorted minimizers
[M::main::3.195*2.27] loaded/built the index for 207 target sequence(s)
[M::mm_mapopt_update::3.456*2.17] mid_occ = 233
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 207
[M::mm_idx_stat::3.626*2.12] distinct minimizers: 13929385 (49.04% are singletons); average occurrences: 2.379; average spacing: 5.333
[M::worker_pipeline::39.153*6.80] mapped 39209 sequences
[M::worker_pipeline::65.777*7.42] mapped 38485 sequences
[M::worker_pipeline::93.197*7.68] mapped 39045 sequences
[M::worker_pipeline::121.159*7.81] mapped 37099 sequences
[M::worker_pipeline::148.646*7.90] mapped 41300 sequences
[M::worker_pipeline::176.643*7.97] mapped 39129 sequences
[M::worker_pipeline::204.333*8.01] mapped 39455 sequences
[M::worker_pipeline::232.592*8.04] mapped 38065 sequences
[M::worker_pipeline::261.818*8.07] mapped 38759 sequences
[M::worker_pipeline::321.433*8.05] mapped 35790 sequences
[M::worker

In [13]:
##generated the read ID list
MappedIdsFn = BamFn.replace('.bam', '.mappedids.txt')
SamtoolsStatsFn = BamFn.replace('.bam', '.stats.txt')
!samtools stats -@ {threads} {BamFn} > {SamtoolsStatsFn}
!samtools  view -F 4  {BamFn} | cut -f 1 | sort | uniq > {MappedIdsFn}

In [15]:
!wc {MappedIdsFn}

  565566   565566 20925942 /media/nvme-02/benWorking/projects/Pgt210Methylation/analysis/germinatedSpores/20210406.Pgt210.PgtGerminatedSpores.sorted.mappedids.txt


In [16]:
#subset the reads
!fast5_subset -i {InFast5Dir} -s {OutDir} -l {MappedIdsFn} -t {threads + 4}

DEBUG:h5py._conv:Creating converter from 5 to 3             |  0% ETA:  --:--:--
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
DEBUG:h5py._conv:Creating converter from 5 to 3
| 565776 of 565776|##########################################|100% Time: 0:59:30
INFO:Fast5Filter:565566 reads extracted


In [ ]:
#basecall the mapped reads only
!guppy_basecaller -i {OutDir} -s {OutDirGuppy} --fast5_out -r --compress_fastq --device auto -c dna_r9.4.1_450bps_hac.cfg

ONT Guppy basecalling software version 4.4.1+1c81d62
config file:        /opt/ont-guppy/data/dna_r9.4.1_450bps_hac.cfg
model file:         /opt/ont-guppy/data/template_r9.4.1_450bps_hac.jsn
input path:         /media/nvme-02/benWorking/projects/Pgt210Methylation/analysis/germinatedSpores/subsetFast5
save path:          /media/nvme-02/benWorking/projects/Pgt210Methylation/analysis/germinatedSpores/subsetFast5/basecalled
chunk size:         2000
chunks per runner:  512
records per file:   4000
fastq compression:  ON
num basecallers:    4
gpu device:         auto
kernel path:        
runners per device: 4
Found 142 fast5 files to process.
Init time: 2335 ms

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
********

In [22]:
print('hello')

hello
